## Imports

In [1]:
import os
from operator import add
from collections import Counter
import io
import numpy as np
import pandas as pd
import regex
import json

from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf
from pyspark.sql.types import *
import pyspark.sql.functions as F
from pyspark.sql.functions import countDistinct, col, udf, stddev, avg


## Configs

In [2]:
PRODIGY_DATASET_PATH = 'data/prodigy/descriptions.jsonl'
TWITTER_PROFILE_PATHS = 'data/*_user_info.txt'
IDENTITY_DICTIONARIES_PATH = 'data/identities/'
PRODIGY_PATTERN_FILE_PATH = 'data/prodigy/patterns.jsonl'

## Initialization

In [3]:
schema = StructType([
    StructField("uid",StringType(),True),
    StructField('name',StringType(),True),
    StructField("screen_name",StringType(),True),
    StructField('url',StringType(),True),
    StructField('protected',StringType(),True),
    StructField('location',StringType(),True),
    StructField('description',StringType(),True),
    StructField("followers_count",IntegerType(),True),
    StructField("friends_count",IntegerType(),True),
    StructField("created_at",StringType(),True),
    StructField("utc_offset",StringType(),True),
    StructField('time_zone',StringType(),True),
    StructField("statuses_count",IntegerType(),True),
    StructField("lang",StringType(),True),
    StructField("status_created_at",StringType(),True),
    StructField('status_coordinates',StringType(),True),
    StructField("status_lang",StringType(),True),
    StructField("profile_image_url_https",StringType(),True),
    StructField("verified",StringType(),True)
])

spark = SparkSession.builder.appName("spark-app").config("PYSPARK_PYTHON","python").getOrCreate()
profiles = spark.read.csv(TWITTER_PROFILE_PATHS, header=False, sep="\t", schema=schema)

profiles.show(3)

+------------------+----------------+-----------+----+---------+------------+--------------------+---------------+-------------+--------------------+----------+---------+--------------+----+--------------------+------------------+-----------+-----------------------+--------+
|               uid|            name|screen_name| url|protected|    location|         description|followers_count|friends_count|          created_at|utc_offset|time_zone|statuses_count|lang|   status_created_at|status_coordinates|status_lang|profile_image_url_https|verified|
+------------------+----------------+-----------+----+---------+------------+--------------------+---------------+-------------+--------------------+----------+---------+--------------+----+--------------------+------------------+-----------+-----------------------+--------+
|        2590887015|        Jennifer|    jp62783|None|    False|        null|                null|              5|           12|Sun Jun 08 01:41:...|      None|     None|  

## Filtering english and regex matches

In [4]:
def regex_filters(df, regex_list):
    for i, regex in enumerate(regex_list):
        df = df.filter(profiles.description.rlike(regex))
    return df

print("initial df size:", profiles.count())

eng_filtered = profiles.filter(
    (profiles.lang.startswith("en")) &
    (profiles.status_lang.startswith("en"))
)

print("after filtering english profiles:", eng_filtered.count())


initial df size: 1394484
after filtering english profiles: 408237


In [5]:
eng_filtered.filter(profiles.description.isNotNull()).count()

237482

In [6]:
%%time

regex_list = [
    "((\w\s*)+[,|#/]\s*)",
    "(i am a )|(I am a )|(I'm a )|(i'm a )|(i am an )|(I am an )|(I'm an )|(i'm an )",
]

regex_filtered = regex_filters(eng_filtered, regex_list)
print("after applying regex size:", regex_filtered.count())

regex_filtered.select('description').head(5)

after applying regex size: 2554
CPU times: user 11.9 ms, sys: 5.69 ms, total: 17.6 ms
Wall time: 6.78 s


[Row(description='I am an Oakwood Man, author, actor, and producer. Follow me, as I follow Christ!'),
 Row(description='I am whole, I am fearfully and wonderfully made, humane, I am a mother/gmom, helper 2 mankind, singer, teacher, student, servant, leader...'),
 Row(description="I am aware, that I am an asshole. I really don't care, about all of that though."),
 Row(description="17. I Love bands. I Like to take photos of stuff. I'm a nerd. follow my tumblr  http://t.co/K2KehN3X"),
 Row(description="I'm a Christian, working man, and dad")]

## Saving descriptions in prodigy format

In [7]:

descs = list(map(
    lambda x: x.description,
    regex_filtered.select('description').collect()
))

with open(PRODIGY_DATASET_PATH, 'w') as f:
    for desc in descs:
        f.write(json.dumps({'text': desc}) + '\n')
        

## Extracting prodigy patterns from dictionaries

In [8]:

ignored_ids = ['person', 'god', 'a', 'fan']

def extract_prodigy_patterns(identities_dir):
    
    def build_pattern(words, label='IDENTITY'):
        return {
            'label': label,
            'pattern': [{'lower': _id_part.lower()} for _id_part in words.split(' ')]
        }
            
    patterns = []   
    for id_rel_path in os.listdir(identities_dir):
        
        id_path = os.path.join(identities_dir, id_rel_path)
        print('reading file:', id_path)
        
        with open(id_path, 'r') as f:
            
            ids = f.read().split('\n')
            for _id in ids:
                
                if len(_id) == 0 or _id in ignored_ids:
                    continue
                
                pattern = build_pattern(_id)
                patterns.append(json.dumps(pattern))
    return patterns
        
patterns = extract_prodigy_patterns(IDENTITY_DICTIONARIES_PATH)

with open(PRODIGY_PATTERN_FILE_PATH, 'w') as f:
    f.write('\n'.join(patterns))

reading file: data/identities/job_identities.txt
reading file: data/identities/national_identities.txt
reading file: data/identities/uga_identities.txt
reading file: data/identities/identities.txt
reading file: data/identities/racial_slur_identities.txt
reading file: data/identities/wordnet_identities.txt
reading file: data/identities/twitter_identities.txt
